# Introduction

This notebook shows an example of relaxing the LiFePO4 crystal.

If you are running this notebook from Google Colab, uncomment the next code box to install m3gnet first.

> Note: Because Google Colab's python version is sometimes rather old (v3.7 at the time of writing this notebook (Jul 2022), you will need to install m3gnet without dependencies and install the dependencies manually.

In [1]:
# !pip install --no-deps m3gnet
# !pip install pymatgen==2022.7.24.1 ase
# !pip install lxml

In [2]:
import warnings

import numpy as np
from pymatgen.ext.matproj import MPRester
from pymatgen.util.coord import pbc_diff

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")

/Users/shyue/repos/pymatgen/pymatgen/analysis/phase_diagram.py:24: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import tensorflow as tf

tf.config.experimental.list_physical_devices("GPU")

[]

In [4]:
mpr = MPRester()
lfp = mpr.get_structure_by_material_id("mp-19017")  # This is LiFePO4.

lfp_strained = lfp.copy()  # We create a copy.
# Create a random strain between -5% and 5% for each direction
strains = np.random.uniform(low=-0.05, high=0.05, size=3)
lfp_strained.apply_strain(strains)
# In addition to the lattice strains, we also perturb the atoms by a distance of 0.1 angstrom.
lfp_strained.perturb(0.1)

In [5]:
relaxer = Relaxer()
relax_results: dict
%time relax_results = relaxer.relax(lfp_strained)
relaxed = relax_results["final_structure"]

2022-07-24 15:03:14.771265: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


CPU times: user 26.5 s, sys: 52.1 s, total: 1min 18s
Wall time: 13.8 s


Note that the relaxation only took < 20s.

In [6]:
print(f"Original lattice parameters are {[round(x, 3) for x in lfp.lattice.abc]}")
print(f"Strained lattice parameters are {[round(x, 3) for x in lfp_strained.lattice.abc]}")
print(f"Relaxed lattice parameters are {[round(x, 3) for x in relaxed.lattice.abc]}")

Original lattice parameters are [4.746, 10.444, 6.09]
Strained lattice parameters are [4.895, 10.419, 5.992]
Relaxed lattice parameters are [4.75, 10.479, 6.089]


In [7]:
print(f"Diff in fractional coords:\n{pbc_diff(lfp.frac_coords, relaxed.frac_coords)}")

Diff in fractional coords:
[[ 9.36742659e-03 -1.06978158e-04 -7.19906965e-03]
 [-1.13854034e-02 -3.68619181e-03  5.14614062e-03]
 [ 1.50647751e-03  6.10458229e-03  1.90134261e-03]
 [ 7.48656414e-03 -4.61575271e-03  6.84919723e-04]
 [-1.02338781e-02  1.76648595e-05 -3.97704039e-04]
 [ 1.24420090e-03 -8.20538344e-04  4.61019604e-03]
 [-1.27134310e-02 -6.71093940e-04  4.28046841e-03]
 [ 3.59836654e-03  2.88985182e-04  5.91889157e-03]
 [-5.31843108e-03  7.02385796e-04 -2.76238489e-04]
 [-6.55750793e-03  5.71826107e-04  1.17736943e-03]
 [-2.71253330e-03  5.44621346e-04  3.25367010e-03]
 [-1.60554610e-03 -6.55113041e-04  1.90726132e-03]
 [-5.09356981e-03 -1.90120389e-04  2.53356120e-03]
 [-5.70023410e-03  1.86424315e-03 -3.63091794e-03]
 [-3.79238318e-03 -1.40521337e-03  7.29282734e-03]
 [-3.02907960e-03 -1.01625552e-04 -3.41108745e-03]
 [-3.35956380e-03  6.66578351e-05 -3.49700988e-03]
 [-9.54934588e-04 -4.95842832e-04  4.95638018e-03]
 [-5.76952829e-03  1.92099513e-04 -3.64330067e-03]
 [-5

Quite clealy, the relaxation using the M3GNet universal IAP has brought the lattice parameters much closer to the original DFT one and the coordinates are also within $10^{-3}$ of the original fractional coordinates.